<center><h1> Phishing detection </h1></center>

<a id='index-of-content'></a>
<h2> 1. Index </h2>

<a id='lectura-de-datos'></a>
<h2> 2. Data reading </h2>

In [43]:
import pandas

urls = pandas.read_csv('tb1_dataset_urls.csv', header=0)

In [44]:
atributos = urls.loc[:, 'qty_dot_url':'email_in_url']  # selección de las columnas de atributos
objetivo = urls['phishing']

<a id='separacion-de-conjuntos'></a>
<h2> 4. Separation of training and testing set </h2>

In [45]:
print(pandas.Series(objetivo).value_counts(normalize=True))

0    0.65428
1    0.34572
Name: phishing, dtype: float64


In [50]:
from sklearn import model_selection

(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Valor de la semilla aleatoria, para que el muestreo sea reproducible,
        # a pesar de ser aleatorio
        random_state=42,
        # Tamaño del conjunto de prueba
        test_size=.33,
        # Estratificamos respecto a la distribución de valores en la variable objetivo
        stratify=objetivo)

<a id='modelos-de-clasificacion'></a>
<h2> 4. Classification models </h2>

<a id='knn-5-vecinos'></a>
<h2> 4.1. KNN Classifier </h2>

In [51]:
from sklearn import neighbors
def score_kNN(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba, k):    
    clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=k, metric='hamming')
    clasif_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)

    return clasif_kNN.score(atributos_prueba, objetivo_prueba)

In [53]:
score_kNN = score_kNN(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba, 5)

In [55]:
import urllib.parse as urlparse
def get_length_top_level_domain(url):
    url = urlparse.urlparse(url)
    domain = url.netloc
    domain = domain.split('.')
    return (0, len(domain[-1]))[len(domain)>1]

In [59]:
import re
def check_is_email_present(url):
    if re.match(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', url):
        return 1
    else:
        return 0

In [60]:
def obtain_tb1_attributes(url):
    qty_dot = url.count('.')
    qty_hyphen = url.count('-')
    qty_underline = url.count('_')
    qty_slash = url.count('/')
    qty_questionmark = url.count('?')
    qty_equal = url.count('=')
    qty_at = url.count('@')
    qty_and = url.count('&')
    qty_exclamation = url.count('!')
    qty_space = url.count(' ')
    qty_tilde_url = url.count('~')
    qty_comma_url = url.count(',')
    qty_plus_url = url.count('+')
    qty_asterisk_url = url.count('*')
    qty_hash_url = url.count('#')
    qty_dollar_url = url.count('$')
    qty_percent_url = url.count('%')
    qty_length_top_level_domain = get_length_top_level_domain(url)
    length = len(url)
    is_email_present = check_is_email_present(url)
    return [qty_dot, qty_hyphen, qty_underline, qty_slash, qty_questionmark, qty_equal, qty_at, qty_and, qty_exclamation, qty_space, qty_tilde_url, qty_comma_url, qty_plus_url, qty_asterisk_url, qty_hash_url, qty_dollar_url, qty_percent_url, qty_length_top_level_domain, length, is_email_present]
    

In [65]:
clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=5, metric='hamming')
clasif_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)
incoming_url = obtain_tb1_attributes("hla")
prediction = clasif_kNN.predict([incoming_url])
print(prediction)

[0]


<a id='decision-tree'></a>
<h2> 4.2. Decission Tree Classifier</h2>

In [66]:
from sklearn.tree import DecisionTreeClassifier
def score_tree_classifier(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clasif_Tree = DecisionTreeClassifier(random_state=0)
    clasif_Tree.fit(atributos_entrenamiento, objetivo_entrenamiento)

    return clasif_Tree.score(atributos_prueba, objetivo_prueba)

In [67]:
score_tree_classifier(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

0.9026457920284405

In [70]:
clasif_Tree = DecisionTreeClassifier(random_state=0)
clasif_Tree.fit(atributos_entrenamiento, objetivo_entrenamiento)
incoming_url = obtain_tb1_attributes("https://google.com")
prediction = clasif_Tree.predict([incoming_url])
print(prediction)

[0]


<a id='neuronal-network'></a>
<h2> 4.3. Neural Network Classifier</h2>

In [46]:
from sklearn.neural_network import MLPClassifier
def score_neuronal_network(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 5), random_state=1)
    clf.fit(atributos_entrenamiento, objetivo_entrenamiento)
    
    return clf.score(atributos_prueba, objetivo_prueba)

In [47]:
score_neuronal_network(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

/Users/maribelrb/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.634921720106652

<a id='random-forest'></a>
<h2>4.4. Random Forest Classifier</h2>

In [50]:
from sklearn.ensemble import RandomForestClassifier
def score_random_forest(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clasif_RF = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    clasif_RF.fit(atributos_entrenamiento, objetivo_entrenamiento)
    return clasif_RF.score(atributos_prueba, objetivo_prueba)

In [51]:
score_random_forest(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

0.8642578792643741

<a id='logistic-regresion'></a>
<h2>4.5. Logistic Regression Classifier</h2>

In [52]:
from sklearn.linear_model import LogisticRegression
def score_Logistic_Regression(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba):
    clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
    clf.fit(atributos_entrenamiento, objetivo_entrenamiento)
    return clf.score(atributos_prueba, objetivo_prueba)

In [53]:
score_Logistic_Regression(atributos_entrenamiento, objetivo_entrenamiento,atributos_prueba, objetivo_prueba)

/Users/maribelrb/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8836398441238805